In [18]:
import pandas as pd

NPYD_collions = pd.read_csv('NYPD_Motor_Vehicle_Collisions.csv',delimiter = ',')

In [26]:
NYPD_df = pd.DataFrame(NPYD_collions)

In [420]:
#Q1

NYPD_df = NYPD_df.set_index(NYPD_df['UNIQUE KEY'])

NYPD_df[(NYPD_df['DATE']<'01/01/2019')]

NYPD_df_unique = NYPD_df.drop_duplicates(keep='first')


In [131]:
NYPD_df_q1_1 = NYPD_df_unique[['NUMBER OF PERSONS INJURED']].sum()

pd.set_option('precision', 0)

print(NYPD_df_q1)

NUMBER OF PERSONS INJURED    38593
dtype: float64


In [159]:
#Q2
NYPD_df_unique['DATE'] = pd.to_datetime(NYPD_df_unique['DATE'])

NYPD_df_q2=NYPD_df_unique[(NYPD_df_unique['DATE']>'2015-12-31') & (NYPD_df_unique['DATE']<'2017-01-01')]

NYPD_df_q2=NYPD_df_q2[NYPD_df_q2['BOROUGH'].notnull()]

NYPD_df_q2_b = NYPD_df_q2[NYPD_df_q2['BOROUGH']=='BROOKLYN']

Q2_Ratio = NYPD_df_q2_b['UNIQUE KEY'].count()/NYPD_df_q2['UNIQUE KEY'].count()

print(Q2_Ratio.round(10))


0.3096198006


In [158]:
#Q3
NYPD_df_q3=NYPD_df_unique[(NYPD_df_unique['DATE']>'2015-12-31') & (NYPD_df_unique['DATE']<'2017-01-01')]

NYPD_df_q3_cyc = NYPD_df_q3[(NYPD_df_q3['NUMBER OF CYCLIST INJURED']!=0 )| (NYPD_df_q3['NUMBER OF CYCLIST KILLED']!=0)]

Q3_Ratio = NYPD_df_q3_cyc['UNIQUE KEY'].count()/NYPD_df_q3['UNIQUE KEY'].count()

print(Q3_Ratio.round(10))


0.0216548369


In [266]:
#Q4: For each borough, compute the number of accidents per capita involving alcohol in 2017.
'''
Step 1: Data Prep
Found out records:in 2017, BOROUGH is not null, contributing factor is "Alcohol Involvement"
'''
NYPD_df_unique['DATE'] = pd.to_datetime(NYPD_df_unique['DATE'])

NYPD_df_q4=NYPD_df_unique[(NYPD_df_unique['DATE']>'2016-12-31') & (NYPD_df_unique['DATE']<'2018-01-01')].copy()

NYPD_df_q4=NYPD_df_q4[NYPD_df_q4['BOROUGH'].notnull()]

NYPD_df_q4_ach = NYPD_df_q4[(NYPD_df_q4['CONTRIBUTING FACTOR VEHICLE 1']=='Alcohol Involvement')]

'''
Step 2 Grouping
Group records by BOROUGH, adding Population to grouped dataset
'''
NYPD_df_q4_ach = NYPD_df_q4_ach.groupby(['BOROUGH']).count()

BOROUGH_POP = pd.DataFrame ({'BOROUGH':['BRONX','BROOKLYN','MANHATTAN','QUEENS','STATEN ISLAND'],
                             'POP':[1471160,2648771,1664727,2358582,479458]})

NYPD_df_q4_ach = NYPD_df_q4_ach.join(BOROUGH_POP.set_index('BOROUGH'), on ='BOROUGH')

'''
Step 3 Calc
Calculate Achohol Involed Accidents per Capita, find the highest rate BOROUGH
Print highest rate BOROUGH
'''

NYPD_df_q4_ach['r']= NYPD_df_q4_ach['UNIQUE KEY']/NYPD_df_q4_ach['POP']

NYPD_df_q4_result = NYPD_df_q4_ach['r'].max()

print(NYPD_df_q4_result.round(10))


0.0002133065


In [265]:
#Q5: Obtain the number of vehicles involved in each collision in 2016.
'''
Step 1 Data Cleaning
Get records in 2016
'''
NYPD_df_q5=NYPD_df_unique[(NYPD_df_unique['DATE']>'2015-12-31') & (NYPD_df_unique['DATE']<'2017-01-01')].copy()

'''
Step 2 Cars Count
Find cars related columns, stack, and count cars in Unique Key
'''
NYPD_df_q5_car= NYPD_df_q5.copy()

NYPD_df_q5_car.drop(NYPD_df_q5_car.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]],axis =1, inplace = True)

car = NYPD_df_q5_car.stack()

car = pd.DataFrame(car.groupby(['UNIQUE KEY']).count())
car.columns = ['CARS']

'''
Step 3 Find car count by zipcode
Join Main record with Car Count
Groupby Zipcode and sum car count
print max
'''

NYPD_df_q5 = NYPD_df_q5.join(car)

NYPD_df_q5=NYPD_df_q5.groupby(['ZIP CODE'])['CARS'].agg('sum')

result = NYPD_df_q5.max()

print(result)




5703.0


In [288]:
#Q6：Year- Collisions Linear regression


'''
Step 1 Data Prep
Find records 2013-2018, covert date into year
'''

import numpy as np

NYPD_df_q6= NYPD_df_unique[(NYPD_df_unique['DATE']>'2012-12-31') & (NYPD_df_unique['DATE']<'2019-01-01')].copy()

NYPD_df_q6['YEAR'] = NYPD_df_q6['DATE'].dt.year

NYPD_df_q6.drop(NYPD_df_q6.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,24,25,26,27,28]],axis =1, inplace = True)

'''
Step 2 Sum Data
Convert data into year- count of collisions
'''
NYPD_df_q6=NYPD_df_q6.groupby(['YEAR']).count()

NYPD_df_q6['Y']= NYPD_df_q6.index

'''
Step 3 Slope
Calc slope use numpy
'''
slopes = NYPD_df_q6.apply(lambda x: np.polyfit(NYPD_df_q6['Y'],NYPD_df_q6['UNIQUE KEY'],1)[0])

print(slopes[0].round(10))

6448.1714285721


In [350]:
#Q7：Winter Driving

'''
Step 1 Data Prep
Find records in 2017, convert dates into month
'''

NYPD_df_q7= NYPD_df_unique[(NYPD_df_unique['DATE']>'2016-12-31') & (NYPD_df_unique['DATE']<'2018-01-01')].copy()

NYPD_df_q7['MONTH'] = NYPD_df_q7['DATE'].dt.month

'''
Step 2 Multi Vehicle Collisions
Find out all records with 3 or more cars - if VEHICLE TYPE CODE 3 is not null
Group all collisions by month
Group Multi Vehicle Collisions by month
'''

NYPD_df_q7_multi = NYPD_df_q7[NYPD_df_q7['VEHICLE TYPE CODE 3'].notnull()].copy()
NYPD_df_q7.drop(NYPD_df_q7.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,24,25,26,27,28]],axis =1, inplace = True)
NYPD_df_q7_multi.drop(NYPD_df_q7_multi.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,24,25,26,27,28]],axis =1, inplace = True)


NYPD_df_q7=NYPD_df_q7.groupby(['MONTH']).count()

NYPD_df_q7_multi = NYPD_df_q7_multi.groupby(['MONTH']).count()


'''
Step 3 Multi Vehicle Collisions Rate Calculation
Join main table and multi vehicle collisions table, calculate rate by month
'''

NYPD_df_q7_multi['COUNT'] = NYPD_df_q7_multi['UNIQUE KEY']

NYPD_df_q7_multi.drop(NYPD_df_q7_multi.columns[[0]],axis =1, inplace = True)

NYPD_df_q7 = NYPD_df_q7.join(NYPD_df_q7_multi)

NYPD_df_q7['RATE'] = NYPD_df_q7['COUNT']/NYPD_df_q7['UNIQUE KEY']

NYPD_df_q7.drop(NYPD_df_q7.columns[[0,1]],axis =1, inplace = True)

print(NYPD_df_q7)


'''
Step 3 Calculate chi_squared_stat
'''

chi_squared_stat = (((NYPD_df_q7.loc[1,['RATE']]-NYPD_df_q7.loc[5,['RATE']])**2)/NYPD_df_q7.loc[5,['RATE']])

print(chi_squared_stat.round(10))





          RATE
MONTH         
1      0.35394
2      0.06492
3      0.04753
4      0.05334
5      0.05406
6      0.05400
7      0.05512
8      0.05519
9      0.05851
10     0.05772
11     0.05981
12     0.06016
RATE    1.6633
dtype: float64


In [418]:
#Q8: For each zip code, find the highest value for collisions per square kilometer
'''
Step 1 Data Prep & Cleaning
'''

NYPD_df_q8= NYPD_df_unique[(NYPD_df_unique['DATE']>'2016-12-31') & (NYPD_df_unique['DATE']<'2018-01-01')].copy()

NYPD_df_q8=NYPD_df_q8[NYPD_df_q8['ZIP CODE'].notnull()]
NYPD_df_q8=NYPD_df_q8[NYPD_df_q8['LATITUDE'].notnull()]
NYPD_df_q8=NYPD_df_q8[NYPD_df_q8['LONGITUDE'].notnull()]

NYPD_df_q8=NYPD_df_q8[NYPD_df_q8['LATITUDE']>0]
NYPD_df_q8=NYPD_df_q8[NYPD_df_q8['LONGITUDE']<0]

'''
Step 2 Each zip code LATITUDE RANGE & CENTER
'''
q8_lowLA = NYPD_df_q8.copy()

q8_lowLA = q8_lowLA.groupby(['ZIP CODE'], sort=False)['LATITUDE'].min()

q8_lowLA= pd.DataFrame(q8_lowLA)
q8_lowLA['LowLA']=q8_lowLA['LATITUDE']
q8_lowLA.drop(q8_lowLA.columns[[0]],axis =1, inplace = True)

q8_HighLA = NYPD_df_q8.copy()

q8_HighLA = q8_HighLA.groupby(['ZIP CODE'], sort=False)['LATITUDE'].max()

q8_HighLA= pd.DataFrame(q8_HighLA)

q8_la = q8_HighLA.join(q8_lowLA)

q8_la['LaCenter'] = (q8_la['LATITUDE'] + q8_la['LowLA'])/2
q8_la['LaRange'] = (q8_la['LATITUDE']-q8_la['LowLA'])*69+1

q8_la.drop(q8_la.columns[[0,1]],axis =1, inplace = True)

'''
Step 3 Each zip code LONGITUDE RANGE & CENTER
'''
q8_lowLO = NYPD_df_q8.copy()

q8_lowLO = q8_lowLO.groupby(['ZIP CODE'], sort=False)['LONGITUDE'].min()

q8_lowLO= pd.DataFrame(q8_lowLO)
q8_lowLO['LowLO']=q8_lowLO['LONGITUDE']
q8_lowLO.drop(q8_lowLO.columns[[0]],axis =1, inplace = True)

q8_HighLO = NYPD_df_q8.copy()

q8_HighLO = q8_HighLO.groupby(['ZIP CODE'], sort=False)['LONGITUDE'].max()

q8_HighLO= pd.DataFrame(q8_HighLO)

q8_lo = q8_HighLO.join(q8_lowLO)

q8_lo['LoCenter'] = (q8_lo['LONGITUDE'] + q8_lo['LowLO'])/2
q8_lo['LoRange'] = (q8_lo['LONGITUDE'] - q8_lo['LowLO'])*69+1

q8= q8_la.join(q8_lo)


'''
Step 4 Calculate Area for each zip code
'''

q8['AREA'] = q8['LaRange']*q8['LoRange']

q8.drop(q8.columns[[0,1,2,3,4,5]],axis =1, inplace = True)

NYPD_df_q8 = NYPD_df_q8.groupby(['ZIP CODE'])['UNIQUE KEY'].count()

NYPD_df_q8=pd.DataFrame(NYPD_df_q8)

NYPD_df_q8 = NYPD_df_q8.join(q8)

NYPD_df_q8['Per MileSquare'] = NYPD_df_q8['UNIQUE KEY']/NYPD_df_q8['AREA']

'''
Step 5 Print Result
'''

result = NYPD_df_q8.nlargest(10,'Per MileSquare')

result.drop(result.columns[[0,1]],axis =1, inplace = True)

print(result)











          Per MileSquare
ZIP CODE                
10013.0        345.86687
10022.0        334.04975
11203.0        220.94891
11372.0        218.75860
10036.0        209.96594
10065.0        209.39712
11233.0        181.01647
10010.0        163.83497
11230.0        156.89706
11235.0        146.78618
